This cleaning file results in one combined file (poverty.csv) containing data on poverty.    

Data source: https://www.census.gov/programs-surveys/saipe/data/datasets.html

In [42]:
import os
import sys
import requests
import pandas as pd


current_dir =  os.getcwd()

grandparent_dir = os.path.dirname(os.path.dirname(current_dir))
sys.path.insert(0, grandparent_dir)

In [17]:

dataframes = []

for year in range(1998, 2021 + 1):
 
    api_url = f"https://api.census.gov/data/timeseries/poverty/saipe?get=YEAR,GEOID,SAEMHI_PT,SAEPOV0_17_PT,SAEPOVRT0_17_PT,SAEPOVALL_PT,SAEPOVRTALL_PT&for=county:*&in=state:*&time={year}"
    

    response = requests.get(api_url)
    
   
    if response.status_code == 200:
       
        data = response.json()
        columns = data[0]
        data = data[1:]
        df = pd.DataFrame(data, columns=columns)
        dataframes.append(df)
        print(f"DataFrame for year {year} created successfully.")

poverty_raw = pd.concat(dataframes, ignore_index=True)

DataFrame for year 1998 created successfully.
DataFrame for year 1999 created successfully.
DataFrame for year 2000 created successfully.
DataFrame for year 2001 created successfully.
DataFrame for year 2002 created successfully.
DataFrame for year 2003 created successfully.
DataFrame for year 2004 created successfully.
DataFrame for year 2005 created successfully.
DataFrame for year 2006 created successfully.
DataFrame for year 2007 created successfully.
DataFrame for year 2008 created successfully.
DataFrame for year 2009 created successfully.
DataFrame for year 2010 created successfully.
DataFrame for year 2011 created successfully.
DataFrame for year 2012 created successfully.
DataFrame for year 2013 created successfully.
DataFrame for year 2014 created successfully.
DataFrame for year 2015 created successfully.
DataFrame for year 2016 created successfully.
DataFrame for year 2017 created successfully.
DataFrame for year 2018 created successfully.
DataFrame for year 2019 created su

In [18]:
poverty_raw.rename(columns={'YEAR': 'Year', 'GEOID': 'GeoFIPS', 'SAEMHI_PT': 'medianHouseholdIncome',
'SAEPOV0_17_PT': 'povertyUnder18', 'SAEPOVRTALL_PT':'povertyAllprct', 'SAEPOVALL_PT':'povertyAll',
'SAEPOVRT0_17_PT': 'povertyUnder18prct'
    }, inplace=True)

columns_to_keep = ['GeoFIPS', 'Year', 'povertyAll', 'povertyAllprct', 
        'povertyUnder18', 'povertyUnder18prct', 'medianHouseholdIncome']
poverty_raw = poverty_raw.loc[:, columns_to_keep]

In [47]:
from cities.utils.data_grabber import DataGrabber

data = DataGrabber()
data.get_features_wide(["gdp"])
gdp = data.wide["gdp"]

import numpy as np

In [28]:
poverty = poverty_raw.copy()

poverty['GeoFIPS'] = poverty['GeoFIPS'].astype(int)

poverty = poverty.merge(gdp[['GeoFIPS', 'GeoName']], on='GeoFIPS', how='left', suffixes=('_poverty', '_gdp'))


In [60]:
desired_order = ['GeoFIPS', 'GeoName', 'Year', 'povertyAll', 'povertyAllprct',
                  'povertyUnder18', 'povertyUnder18prct', 'medianHouseholdIncome' ]

poverty = poverty[desired_order]


poverty = 

poverty

,GeoFIPS,GeoName,Year,povertyAll,povertyAllprct,povertyUnder18,povertyUnder18prct,medianHouseholdIncome
0,1001,"Autauga, AL",1998,4901,11.3,2149,17.7,38260
1,1003,"Baldwin, AL",1998,15089,11.0,5688,16.8,37162
2,1005,"Barbour, AL",1998,5795,22.7,2183,31.3,27603
3,1007,"Bibb, AL",1998,3313,16.9,1279,23.7,29692
4,1009,"Blount, AL",1998,6176,12.9,2313,19.8,35623
...,...,...,...,...,...,...,...,...
75399,56037,"Sweetwater, WY",2021,3761,9.2,990,9.7,74677
75400,56039,"Teton, WY",2021,1392,5.9,223,5.5,102709
75401,56041,"Uinta, WY",2021,1917,9.4,635,11.2,70162
75402,56043,"Washakie, WY",2021,773,10.2,195,12.5,62176


In [61]:
# save them as seperate files

columns_to_save = poverty.columns[poverty.columns.get_loc("Year") + 1 :]

for column in columns_to_save:

    selected_columns = ["GeoFIPS", "GeoName", "Year", column]
    subsetpoverty = poverty[selected_columns]
    wide_df = subsetpoverty.pivot(index=['GeoFIPS', 'GeoName'], columns='Year', values= column).reset_index()

    wide_df = wide_df.fillna(0)

    file_name_wide = f"{column}_wide.csv"
    wide_df.to_csv(f"{grandparent_dir}/data/raw/{file_name_wide}", index=False)

In [62]:
df = pd.read_csv(f"{grandparent_dir}/data/raw/povertyAll_wide.csv")
df.shape[0]

3151

In [50]:
common_fips = np.intersect1d(
    gdp["GeoFIPS"].unique(), df["GeoFIPS"].unique()
)
variable_db = df[df["GeoFIPS"].isin(common_fips)]
variable_db = variable_db.merge(
    gdp[["GeoFIPS", "GeoName"]], on=["GeoFIPS", "GeoName"], how="left"
)




variable_db

,GeoFIPS,GeoName,1998,1999,2000,2001,2002,2003,2004,2005,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,1001,"Autauga, AL",4901.0,4991.0,4681.0,4904.0,4795.0,4926.0,5627.0,5020.0,...,7017.0,7397.0,7204.0,6966.0,7444.0,7390.0,7587.0,6723.0,6242.0,6296.0
1,1003,"Baldwin, AL",15089.0,14668.0,13855.0,16313.0,16175.0,15753.0,16089.0,18249.0,...,26177.0,27429.0,25696.0,25941.0,24005.0,21199.0,21069.0,22360.0,20189.0,25526.0
2,1005,"Barbour, AL",5795.0,5903.0,5847.0,6312.0,6152.0,5717.0,6109.0,5694.0,...,6892.0,6654.0,5943.0,7380.0,6787.0,7414.0,6788.0,5909.0,5548.0,5089.0
3,1007,"Bibb, AL",3313.0,3399.0,3561.0,3605.0,3546.0,3383.0,3476.0,3364.0,...,4371.0,4689.0,3666.0,4516.0,4099.0,4137.0,4400.0,4101.0,3549.0,4204.0
4,1009,"Blount, AL",6176.0,6271.0,6193.0,7152.0,6792.0,6503.0,6844.0,6248.0,...,9264.0,9878.0,10000.0,8399.0,8033.0,7343.0,7527.0,9324.0,7525.0,6992.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3069,56037,"Sweetwater, WY",3200.0,3084.0,2788.0,2741.0,2903.0,3192.0,3028.0,2834.0,...,3741.0,3850.0,4346.0,3744.0,4264.0,4253.0,3540.0,3453.0,3187.0,3761.0
3070,56039,"Teton, WY",722.0,863.0,854.0,1042.0,1037.0,1067.0,928.0,1144.0,...,1885.0,1700.0,1751.0,1516.0,1680.0,1348.0,1443.0,1396.0,1207.0,1392.0
3071,56041,"Uinta, WY",2124.0,2137.0,1916.0,1918.0,1951.0,2063.0,1922.0,2046.0,...,2646.0,2515.0,2081.0,2030.0,2183.0,2067.0,2010.0,1699.0,1697.0,1917.0
3072,56043,"Washakie, WY",907.0,973.0,877.0,836.0,836.0,860.0,837.0,971.0,...,993.0,941.0,1030.0,916.0,1060.0,845.0,918.0,845.0,739.0,773.0


In [65]:
#df =  df.fillna(0)

df[df['GeoFIPS'] == 2158]

,GeoFIPS,GeoName,1998,1999,2000,2001,2002,2003,2004,2005,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
83,2158,"Kusilvak Census Area, AK*",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2576.0,3033.0,3050.0,2905.0,2217.0,2305.0,2534.0
